In [1]:
#https://www.amazon.in/Apple-iPhone-Silver-Storage-Display/product-reviews/B0711T2L8K/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews

import urllib
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from random import randint
import requests

In [2]:

def requestURL(url):
    proxies = {"http": "http://proxy-chain.intel.com:911","https": "http://proxy-chain.intel.com:912"} 
    headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    url_response = requests.get(url,proxies=proxies, headers=headers)    
    page=url_response.content
    status = url_response.status_code
    soup = BeautifulSoup(page)
    BShtml = soup.prettify('utf-8')  
    return soup, status

In [3]:
#https://www.amazon.in/Apple-iPhone-Silver-64GB-Storage/dp/B0711T2L8K/ref=sr_1_1?s=electronics&ie=UTF8&qid=1548335262&sr=1-1&keywords=iphone+X
soup, status = requestURL(url=input('Enter Amazon Product Link: '))

#Goto all reviews link:
allReviewsURL = 'https://www.amazon.in'+ soup.find('a', attrs={'data-hook':'see-all-reviews-link-foot'})['href']

Enter Amazon Product Link: https://www.amazon.in/Apple-iPhone-Silver-64GB-Storage/dp/B0711T2L8K/ref=sr_1_1?s=electronics&ie=UTF8&qid=1548335262&sr=1-1&keywords=iphone+X


C:\Users\silkymis\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\silkymis\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
def soup_extractor(URL):
    for retry in range(1,5):
        sleep(randint(1,5))
        soupAll, status = requestURL(URL)
        if(status!=200):
            soup_extractor(URL)            
        else:
            return soupAll


In [5]:
reviewer = []
reviewDate = []
colour = []
size=[]
verifiedPurchase = []
rating = []
reviewTitle=[]
reviewDesc= []


In [9]:
def review_Scraper(reviewURL):
    for i in range(2,111):
        sleep(randint(1,6))
      
        
        soupAll = soup_extractor(reviewURL)  
                 
        reviews= soupAll.find('div',attrs={'id':'cm_cr-review_list'})
        if(reviews.find_all('div', attrs={'data-hook':'review'}) is not None):
            
            for div in reviews.find_all('div', attrs={'data-hook':'review'}):
            
                reviewer.append(div.find('span', attrs={'class':'a-profile-name'}).text.strip())    

                reviewDate.append(div.find('span', attrs={'data-hook':'review-date'}).text.strip())

                details = div.find('a', attrs={'class': 'a-size-mini a-link-normal a-color-secondary'}).text.split(':')
                colour.append((re.sub('Size', '' ,details[1])).strip())
                size.append(details[2].strip())

                if(div.find('span', attrs={'data-hook':'avp-badge'})):
                    verifiedPurchase.append(True)
                else:
                    verifiedPurchase.append(False)


                rating.append(div.find('i', attrs={'data-hook':'review-star-rating'}).span.text.split()[0])


                reviewTitle.append(div.find('a', attrs={'data-hook':'review-title'}).text.strip())

                reviewDesc.append(div.find('span', attrs={'data-hook':'review-body'}).text.strip())
            else:
                continue
        if((soupAll.find('li', attrs={'class':'a-last'}).a.get('href')) is None):
            break;
        else:
            reviewURL= 'https://www.amazon.in'+soupAll.find('li', attrs={'class':'a-last'}).a['href']


In [10]:

review_Scraper(allReviewsURL)

C:\Users\silkymis\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\silkymis\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [11]:
import numpy as np
reviews_df= pd.DataFrame()
reviews_df['Reviewer'] = pd.Series(reviewer, dtype=str)
reviews_df['DateOfReview'] = pd.Series(reviewDate, dtype=np.datetime64)
reviews_df['Colour'] = pd.Series(colour, dtype=str)
reviews_df['Size'] = pd.Series(size, dtype=str)
reviews_df['VerifiedPurchase'] = pd.Series(verifiedPurchase, dtype=bool)
reviews_df['Rating'] = pd.Series(rating, dtype=float)
reviews_df['ReviewTitle'] = pd.Series(reviewTitle, dtype=str)
reviews_df['ReviewDescription'] = pd.Series(reviewDesc, dtype=str)

C:\Users\silkymis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  after removing the cwd from sys.path.
C:\Users\silkymis\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
reviews_df.to_csv('AmazonReviews.csv', index=False)